# MindLogger Survey Setup

MindLogger is a mobile application for survey data collection, including voice, video, and drawing. We are very much in the development stage, so here are temporary instructions to set up your own MindLogger survey.

One main purpose of the MindLogger project is to promote reusability of individual survey questions. This way, if two people, or studies, collect data from similar questions, the data can easily be combined.

In order for this to work, we must complete the following steps:

1. Create a set of surveys (called an applet) following our schema, and post this survey to GitHub
2. Import your survey on the Mindlogger web app
3. On the admin panel, set your notification schedule
4. Download the mindlogger app to your iOS or Android device

Steps:

1. `git clone https://github.com/akeshavan/mindlogger-build-applet`
2. create a new github repository to host your files
3. get a [github personal access token](https://help.github.com/en/articles/creating-a-personal-access-token-for-the-command-line) with repo scope and save it as a system variable called "GH_TOKEN" `export GH_TOKEN=`
4. run `python setup.py install` at the root of this repo (or `python setup.py develop`) to install this package
5. Open this notebook with Python3 on your computer
6. in your github repository, make the **"master"** branch the one that renders to github pages. Also click the **enforce https** checkbox:

<img src="./githubpagesSettings.png" alt="drawing" width="50%"/>



Our goal is to be able to set up a MindLogger applet without any programming knowledge. But for now, this is the best way.

In [1]:
import mindlogger_build_applet as mba

First, let's set up the "storage". Two options:

1. **GitHubStorage** - Read and write in a target repository using a GitHub access token.
2. **LocalStorage** - Read and write in a local filesystem directory.

In [2]:
# Setup for LocalStorage: define `repo_name` and `uri_base` and `local_dir` appropriately.

repo_name = 'ml-test-survey'
base_uri='https://www.github.com/madprime/{}/'.format(repo_name)

import os
from mindlogger_build_applet.utils import LocalStorage

base_dir = os.getcwd()
local_dir = os.path.join(base_dir, repo_name)
if not os.path.exists(local_dir):
    os.mkdir(local_dir)

storage = LocalStorage(
    local_dir=local_dir,
)

print("Writing to: {}".format(local_dir))

Writing to: /home/mad/projects/mindlogger-surveys/ml-test-survey


In [3]:
q1 = mba.Radio(
    # initialize with parameters needed for all nodes
    node_id='hangryNow',
    storage=storage,
    base_uri=base_uri
)

q1.set_data(
    # Radio setup
    prefLabel="asking if i'm hangry now",
    altLabel="hangry",
    description="asking if the user is hangry, yes or no",
    question="are you hangry right now?",
    responseOptions={
        'minValue': 0,
        'maxValue': 1,
        'required': True,
        'type': 'xsd:boolean',
        'choices': [
            {
                'name': 'Yes',
                'value': 1,
            },
            {
                'name': 'No',
                'value': 0,
            },
        ]
    }
)

q2 = mba.Radio(
    node_id="ateWhen",
    storage=storage,
    base_uri=base_uri
)

q2.set_data(
    prefLabel="when you last ate",
    altLabel="ateWhen",
    description="How long ago did you last eat",
    question="How long ago did you last eat?",
    responseOptions={
        'minValue': 0,
        'maxValue': 3,
        'required': True,
        'type': 'xsd:boolean',
        'choices': [
            {
                'name': 'Just now',
                'value': 0,
            },
            {
                'name': '1-2 hours ago',
                'value': 1,
            },
            {
                'name': '2-3 hours ago',
                'value': 2,
            },
            {
                'name': '3+ hours ago',
                'value': 3,
            },
        ]
    }
)
print(q1.data["@id"])


hangryNow


In [4]:
q1.write()
q2.write()

# These are predicted based on `base_uri` setting.
print(q1.get_uri())
print(q2.get_uri())

print(q1.data["@id"])
print(q2.data["@id"])

https://www.github.com/madprime/ml-test-survey/items/hangryNow.jsonld
https://www.github.com/madprime/ml-test-survey/items/ateWhen.jsonld
hangryNow
ateWhen


In [5]:
activity1 = mba.Activity(
    node_id="hangryActivity",
    storage=storage,
    base_uri=base_uri,
)

activity1.set_data(
                prefLabel="hangryness activity",
                altLabel="hangryActivity",
                description="an activity that asks if you're hangry"
            )
activity1.setItem(q1)

# Make the second question conditional if the first question is answered true
activity1.setItem(q2, "hangryNow")

activity1.write()
activity1.context.write()

In [6]:
activitySet = mba.ActivitySet(
    node_id='hangryActivitySet',
    storage=storage,
    base_uri=base_uri,
)

activitySet.set_data(
    prefLabel="hangryness activity set",
    altLabel="hangryActivity",
    description="an activitySet that asks if you're hangry, and what you ate and when"
)

# second argument is the label of the activity on the sidebar of mindlogger-web
activitySet.setActivity(activity1, 'Hangry')
activitySet.write()
activitySet.context.write()

In [7]:
activitySet.preview()

In [8]:
activitySet.mindlogger()

At this point, you have added your applet to MindLogger via the web-interface. You can take your applet on the web interface, or you can take it on your phone.

### Download MindLogger to your phone

Follow the instructions on the mindlogger.org website for downloading to iOS or Android: https://mindlogger.org/download/


### Setting up Notifications on mobile

You can set up notifications on your phone through the MindLogger Admin panel: https://admin.mindlogger.org


#### Step 1

Set the server endpoint as https://api.mindlogger.org/api/v1 and then click `continue`

<img src="./adminStep1.png" alt="drawing" width="75%"/>

#### Step 2

Log into your mindlogger account, click `log in` and then `continue`

<img src="./adminStep2.png" alt="drawing" width="75%"/>

#### Step 3

Select your applet, by clicking the dropdown until the it is open and says `CURRENT APPLET` in red. This is a bug, but we will fix it eventually.

<img src="./adminStep3.png" alt="drawing" width="75%"/>

#### Step 4

Here is a google-maps like interface for scheduling. On the month view, set up an all day activity, and set the title to your applet. Then fill in the following fields highlighted in red. You can click the `+` sign (not shown) to add more notifications in a single day.

<img src="./adminStep4.png" alt="drawing" width="75%"/>

note: the things crossed out in gray are not implemented yet.

#### Step 5

Save your schedule by clicking continue in the stepthrough. **This is how the schedule is saved** for right now.

<img src="./adminStep5.png" alt="drawing" width="75%"/>

#### Step 6

To load the notifications in your app, you need to **log out of mindlogger on your phone and log back in** to set up notifications. This is also a bug, and for now this workaround works.

### Updating your applet

As we build these applets, we need to test them out and then update them. To update your applet, you need to re-invite yourself on mindlogger-web (making sure the URL to the applet is the same), then redo the schedule, and log out and then log in to your applet. 
